In [33]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm

### Import data

In [34]:
#training
nn_out = pd.read_csv('roughfeaturedata/preds_for_logreg.csv')
train_features = pd.read_csv('data/train_features.csv')
train_sims = pd.read_csv('roughfeaturedata/train_with_sim_and_ents_long.csv')
tfidf_train_features = pd.read_csv('data/tfidf_train_features.csv')
topic_modelling_train = pd.read_csv('roughfeaturedata/topic_modelling_output.csv')

#test
nn_out_test = pd.read_csv('roughfeaturedata/test_preds_for_logreg.csv')
test_features = pd.read_csv('data/test_features.csv')
tfidf_test_features = pd.read_csv('data/tfidf_test_features.csv')
test_sims = pd.read_csv('roughfeaturedata/test_with_sim_and_ents_long.csv')
topic_modelling_test = pd.read_csv('roughfeaturedata/topic_modelling_output.csv')
topic_modelling_test.columns = ['test_id'] + list(topic_modelling_test.columns[1:])

### Build Complete Test + Training sets

In [35]:
#build training

train = nn_out.merge(train_features, on = 'id')
train = train.merge(train_sims, on = 'id')
train = train.merge(tfidf_train_features, on = 'id')
train = train.merge(topic_modelling_train, on = 'id')


In [36]:
#build test

test = nn_out_test.merge(test_features, on = 'test_id')
test = test.merge(test_sims, on = 'test_id')
test = test.merge(tfidf_test_features, on = 'test_id')
test = test.merge(topic_modelling_test, on = 'test_id')


### Feature list (comment out to exclude)

In [37]:
featurelist = ['similarity_score',
                'nn_out',
                'last_char',
                'avg_shared_words',
                'word_count_diff',
                'levenshtein',
                'shared_words_pcnt',
                'avg_shared_trigrams',
                'shared_bigram_pcnt',
                'shared_trigram_pcnt',
                'avg_shared_quadgrams',
                'shared_quadgram_pcnt',
                'tfidf_word_match_share'] + list(topic_modelling_test.columns[1:])


In [38]:
train = train.fillna(0)
train_x = train[featurelist]
train_y = train['is_duplicate']

test = test.fillna(0)
test_x = test[featurelist]
y_labs = test['test_id']

### Spot-check regression

In [39]:
logit_model=sm.Logit(train_y,train_x)
result=logit_model.fit()
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.246996
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:           is_duplicate   No. Observations:               306325
Model:                          Logit   Df Residuals:                   306212
Method:                           MLE   Df Model:                          112
Date:                Wed, 29 Nov 2017   Pseudo R-squ.:                  0.6250
Time:                        20:14:59   Log-Likelihood:                -75661.
converged:                       True   LL-Null:                   -2.0177e+05
                                        LLR p-value:                     0.000
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
similarity_score          -3.3292      0.053    -62.258      0.000      -3.434      

### Final Regression

In [40]:
logreg = LogisticRegression()
logreg.fit(train_x, train_y)
y_pred = logreg.predict(test_x)

In [41]:
predictions = pd.DataFrame({"test_id":y_labs, "is_duplicate":y_pred})
#prediction.to_csv("finalsubmission.csv", index=False)